In [1]:
from multirotor import MultiRotor

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
import pandas as pd

import seaborn as sns

sns.set_style("whitegrid")

In [2]:
class mfac(nn.Module):
    def __init__(self, e1 = 0, e2 = 0, e3 = 0, e4 = 0, e5 = 0):
        super(mfac, self).__init__()
        # Layers of network
        self.lin1 = nn.Linear(5, 10)
        self.lin2 = nn.Linear(10, 1)
        
        # Initializing error
        self.e1, self.e2, self.e3, self.e4, self.e5 = e1, e2, e3, e4, e5
        
    def forward(self, x):
        # Propagation through the network
        self.e5 = self.e4
        self.e4 = self.e3
        self.e3 = self.e2
        self.e2 = self.e1
        self.e1 = x
        
        x = self.lin1(torch.FloatTensor([self.e1, self.e2, self.e3, self.e4, self.e5]))
        x = F.relu(x)
        x = self.lin2(x)
        
        return x

In [3]:
# Simulation timing
time_start = 0
time_stop = 30
num_time_points = 1000
time = np.linspace(time_start, time_stop, num_time_points)
time_step = time[1] - time[0]

In [4]:
mr = MultiRotor(dt=time_step)
net = mfac()

In [5]:
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.0011)

In [6]:
# Altitude target (setpoint) is up 10m
altitude_setpoint = mr.get_altitude() + 10

In [7]:
for t in time:
    error = float(altitude_setpoint - mr.get_altitude())
    output = net(error)
    curr_altitude, curr_velocity = mr.step(output)
    
    optimizer.zero_grad()
    loss = criterion(output, torch.as_tensor([error]))
    loss.backward()
    optimizer.step()
    
    print(curr_altitude)

9.971309852751768
tensor([9.9411], grad_fn=<AddBackward0>)
tensor([9.9732], grad_fn=<AddBackward0>)
tensor([10.0197], grad_fn=<AddBackward0>)
tensor([9.9977], grad_fn=<AddBackward0>)
tensor([10.0677], grad_fn=<AddBackward0>)
tensor([9.9236], grad_fn=<AddBackward0>)
tensor([9.8468], grad_fn=<AddBackward0>)
tensor([9.8883], grad_fn=<AddBackward0>)
tensor([10.0078], grad_fn=<AddBackward0>)
tensor([9.8428], grad_fn=<AddBackward0>)
tensor([9.8457], grad_fn=<AddBackward0>)
tensor([9.7829], grad_fn=<AddBackward0>)
tensor([9.6938], grad_fn=<AddBackward0>)
tensor([9.6760], grad_fn=<AddBackward0>)
tensor([9.6105], grad_fn=<AddBackward0>)
tensor([9.3989], grad_fn=<AddBackward0>)
tensor([9.4216], grad_fn=<AddBackward0>)
tensor([9.4012], grad_fn=<AddBackward0>)
tensor([9.2900], grad_fn=<AddBackward0>)
tensor([9.2225], grad_fn=<AddBackward0>)
tensor([8.9139], grad_fn=<AddBackward0>)
tensor([8.8472], grad_fn=<AddBackward0>)
tensor([8.5983], grad_fn=<AddBackward0>)
tensor([8.3825], grad_fn=<AddBackwar

tensor([19.8697], grad_fn=<AddBackward0>)
tensor([20.0687], grad_fn=<AddBackward0>)
tensor([20.1353], grad_fn=<AddBackward0>)
tensor([20.1786], grad_fn=<AddBackward0>)
tensor([20.1236], grad_fn=<AddBackward0>)
tensor([19.9520], grad_fn=<AddBackward0>)
tensor([19.8874], grad_fn=<AddBackward0>)
tensor([19.8763], grad_fn=<AddBackward0>)
tensor([19.8907], grad_fn=<AddBackward0>)
tensor([19.6419], grad_fn=<AddBackward0>)
tensor([19.8052], grad_fn=<AddBackward0>)
tensor([19.8421], grad_fn=<AddBackward0>)
tensor([19.6828], grad_fn=<AddBackward0>)
tensor([19.7362], grad_fn=<AddBackward0>)
tensor([19.7073], grad_fn=<AddBackward0>)
tensor([19.6326], grad_fn=<AddBackward0>)
tensor([19.6312], grad_fn=<AddBackward0>)
tensor([19.3584], grad_fn=<AddBackward0>)
tensor([19.6093], grad_fn=<AddBackward0>)
tensor([19.6372], grad_fn=<AddBackward0>)
tensor([19.4214], grad_fn=<AddBackward0>)
tensor([19.4514], grad_fn=<AddBackward0>)
tensor([19.4150], grad_fn=<AddBackward0>)
tensor([19.3595], grad_fn=<AddBack

tensor([19.4783], grad_fn=<AddBackward0>)
tensor([19.5734], grad_fn=<AddBackward0>)
tensor([19.6535], grad_fn=<AddBackward0>)
tensor([19.6370], grad_fn=<AddBackward0>)
tensor([19.5876], grad_fn=<AddBackward0>)
tensor([19.5111], grad_fn=<AddBackward0>)
tensor([19.4754], grad_fn=<AddBackward0>)
tensor([19.5399], grad_fn=<AddBackward0>)
tensor([19.4908], grad_fn=<AddBackward0>)
tensor([19.4735], grad_fn=<AddBackward0>)
tensor([19.3716], grad_fn=<AddBackward0>)
tensor([19.2801], grad_fn=<AddBackward0>)
tensor([19.1765], grad_fn=<AddBackward0>)
tensor([19.0973], grad_fn=<AddBackward0>)
tensor([19.0789], grad_fn=<AddBackward0>)
tensor([19.0461], grad_fn=<AddBackward0>)
tensor([18.7964], grad_fn=<AddBackward0>)
tensor([18.6631], grad_fn=<AddBackward0>)
tensor([18.7951], grad_fn=<AddBackward0>)
tensor([18.7153], grad_fn=<AddBackward0>)
tensor([18.5786], grad_fn=<AddBackward0>)
tensor([18.4010], grad_fn=<AddBackward0>)
tensor([18.3320], grad_fn=<AddBackward0>)
tensor([18.2217], grad_fn=<AddBack

In [ ]:
# frames.append(observation.copy())

# Initialize subplots
fig, ax = plt.subplots()

# Set axis of animation
ax.set(xlim=(0,300), ylim=(240,0))

# Initialize frame
frame = plt.imshow(frames[0])

def init():
    """
    Creates initial frame for animation.
    
    :return: (list) a list of the initial frame for animation
    """
    frame.set_data(frames[0])
    return [frame]

def animate(i):
    """
    Updates the state of the frame at each point of the animation.
    
    :param i: (ndarray) current image/frame
    :return: (list) a list of the current frame for animation
    """
    # Define the updated content of frame
    frame.set_array(i)
    return [frame]

from IPython.display import HTML

# Take images, functions, and other parameters and create an animation
ani = FuncAnimation(fig, animate, frames[::35], init_func = init, interval = 200)
HTML(ani.to_jshtml())
# -

# Save animation
ani.save("animation" + ".gif")